# Genetic Algorithm

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Vissim
import win32com.client as com
import os

#GA
import random
import csv

from deap import base, creator, tools
toolbox = base.Toolbox()

#plot
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors

# Functions Setup

In [ ]:
#設定ga起始解
def load_individuals(creator,param):
    individuals = []
    for i in range(len(param)):
        individual = param[i]
        individual = creator(individual)# <= transform list to Individual, Individual was defined in the previous one, and passed by parameter
        individuals.append(individual)
    return individuals

In [ ]:
#突變方式
def mutUniform(individual, U, L, indpb):
    size = len(individual)
    for i in range(size):
        #individual[i] = random.uniform(U[i], L[i])
        if random.random() < indpb:
            individual[i] = round(random.uniform(U[i], L[i]),2)
    return individual,

In [ ]:
#評估參數績效，以VISSIM模擬後輸出績效
def evaluate(individual,pop,fitnesses):
    if individual not in pop:
        #reset driving behavior
        reset_driving_behavior(individual)

        #run simulation
        # Activate QuickMode:
        Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",1)
        Vissim.Simulation.RunContinuous()

        #save mape value
        F = pd.read_csv(r"C:\Users\hyang\Desktop\HYANG\vissim com\test\new20201107fcal.csv")
        S = pd.read_csv(r"C:\Users\hyang\Desktop\HYANG\vissim com\test\new20201107scal.csv")
        f, s = load_vissim_data()
        f.columns = F.columns
        s.columns = S.columns
        F_mean = round(mape(F.iloc[:,0:20],f.iloc[:,0:20]).mean(),4)*100
        S_mean = round(mape(S.iloc[:,0:20],s.iloc[:,0:20]).mean(),4)*100
        return S_mean,
    else:
        idx = pop.index(individual)
        return fits[idx],

In [ ]:
#設定VISSIM路網中路段駕駛行為參數，需自行調整路段編碼
def reset_driving_behavior(ind):
    #路段編碼
    section = {'front':[102,20002],'mid':[103,20003],'end':[104,20004]}
    #欲調整參數名稱
    pname = ["W99cc0","W99cc1Distr","W99cc2","W99cc7"]
    for pn in range(len(pname)):
        if pn == 0 :
            Vissim.Net.Links.ItemByKey(section['front'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['front'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['mid'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['mid'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])
            Vissim.Net.Links.ItemByKey(section['end'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])
        elif pn == 1:
            Vissim.Net.Links.ItemByKey(section['front'][0]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn])
            Vissim.Net.Links.ItemByKey(section['front'][1]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn])
            Vissim.Net.Links.ItemByKey(section['mid'][0]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['mid'][1]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn+8])
            Vissim.Net.Links.ItemByKey(section['end'][1]).LinkBehavType.DrivBehavDef.W99cc1Distr.SetAttValue("Mean",ind[pn+8])
        elif pn == 2 :
            Vissim.Net.Links.ItemByKey(section['front'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['front'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['mid'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['mid'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])
        elif pn == 3 :
            Vissim.Net.Links.ItemByKey(section['front'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['front'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn])
            Vissim.Net.Links.ItemByKey(section['mid'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['mid'][1]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+4])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])
            Vissim.Net.Links.ItemByKey(section['end'][0]).LinkBehavType.DrivBehavDef.SetAttValue(pname[pn],ind[pn+8])

def mape(train,test):
    result = (abs(train-test)/train).mean()
    return result

def load_vissim_data():
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    for j in range(1,len(Vissim.Net.DataCollectionMeasurements.GetAll())+1):
        temp1 = []
        temp2 = []
        for i in range(1,43):
            i = str(i)
            a = Vissim.Net.DataCollectionMeasurements.ItemByKey(j).AttValue("Vehs(Current,%s,All)"%i)
            b = Vissim.Net.DataCollectionMeasurements.ItemByKey(j).AttValue("SpeedAvgArith(Current,%s,All)"%i)
            if b != None:
                b = round(b,2)
            temp1.append(a)
            temp2.append(b)
        df1[j] = temp1
        df2[j] = temp2
    return df1, df2


In [ ]:
def pull_stats(population, iteration=1):
    fitnesses = [ individual.fitness.values[0] for individual in population ]
    return {
        'i': iteration,
        'mu': np.mean(fitnesses),
        'std': np.std(fitnesses),
        'max': np.max(fitnesses),
        'min': np.min(fitnesses)
    }

# GA

In [ ]:
#載入路網
Net_path = '請自行設定'
Vissim = com.Dispatch("Vissim.Vissim") 
Vissim.LoadNet(r"Net_path")

In [ ]:
#定義問題
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
#using self-defined initial solution
toolbox.register("population", load_individuals,  creator.Individual)
#set fitness
toolbox.register("evaluate", evaluate)
#選擇
toolbox.register("select", tools.selTournament, tournsize=2)
#交配
toolbox.register("mate", tools.cxUniform)
#突變
toolbox.register("mutate", mutUniform, 
                 L=[0.5,2.4,0,0, \
                    0.5,2.3,0,0, \
                    0.5,1.8,0,0],
                 U=[3,2.6,5,1, \
                    3,2.5,5,1,\
                    3,2.5,5,1], indpb=0.08)

In [ ]:
stats = []

In [ ]:
random.seed(64)
# CXPB  is the probability with which two individuals are crossed
# MUTPB is the probability for mutating an individual
CXPB, MUTPB = 0.8, 0.08
#設定起始解，拉丁方格抽樣績效最優的前10組參數
inparam = [
            [0.61, 2.58, 2.14, 0.28, 1.67, 2.55, 1.84, 0.50, 1.60, 2.39, 2.46, 0.08],
            [1.83, 2.40, 2.95, 0.16, 1.75, 2.50, 3.29, 0.81, 2.39, 2.49, 3.11, 0.51],
            [1.08, 2.54, 5.03, 0.65, 2.20, 2.52, 4.97, 0.77, 1.15, 2.20, 4.37, 0.64],
            [0.85, 2.47, 2.93, 0.71, 1.10, 2.53, 1.78, 0.97, 2.48, 2.30, 4.08, 0.37],
            [0.72, 2.51, 5.07, 0.96, 1.12, 2.48, 5.18, 0.79, 0.93, 1.95, 4.31, 0.03],
            [1.19, 2.48, 4.76, 0.66, 2.39, 2.33, 4.63, 0.98, 1.20, 2.47, 4.73, 0.35],
            [0.51, 2.32, 5.75, 0.94, 2.87, 2.56, 5.79, 0.75, 2.44, 2.29, 2.30, 0.19],
            [0.56, 2.37, 4.90, 0.38, 2.17, 2.59, 3.67, 0.86, 2.67, 2.36, 4.98, 0.67],
            [0.58, 2.51, 3.87, 0.18, 1.58, 2.58, 3.57, 0.78, 1.98, 1.97, 5.62, 0.68],
            [0.68, 2.53, 4.70, 0.19, 1.91, 2.53, 0.69, 0.82, 1.80, 2.44, 0.53, 0.82]]
pop = toolbox.population(inparam)

# Evaluate the initial population
fitnesses = [ (f,) for f in pd.read_csv("ga_initial_fitness.csv", index_col=0)['speed mape mean'].round(2).to_list()]
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

# Extracting all the fitnesses of 
fits = [ind.fitness.values[0] for ind in pop]

# Variable keeping track of the number of generations
g = 0
terminal = min(fits)

In [ ]:
# Begin the evolution
# 演化50回合，若在50回合內MAPE績效收斂至15%以下也會停止
while g < 50 and terminal > 15:
    # A new generation
    g = g + 1
    print("-- Generation %i --" % g)
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, 5)
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        toolbox.mate(child1, child2, CXPB)
        del child1.fitness.values
        del child2.fitness.values
    # Apply mutation on the offspring
    for mutant in offspring:
        toolbox.mutate(mutant)
        del mutant.fitness.values
        
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    
    # Input new parameter into vissim
    # Using evaluate function to activate vissim simulation
    # load new fitness
    fitnesses = map(toolbox.evaluate, invalid_ind,pop,fits)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    # Replace the old population by the offspring
    pop = [pop[i] for i in list(np.argsort(fits))]#sort population
    pop[5:] = offspring #replace the last five individual
    for ind in pop:print(ind)
    
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    #print(min(fits), max(fits), mean, std)
    stats.append(pull_stats(pop,g))
    with open('param_history.csv', 'a', newline='') as csvfile:
        # 定義欄位
        colname = ['iteration', 'fCC0', 'fCC1', 'fCC2', 'fCC7', 'mCC0', 'mCC1', 'mCC2', 'mCC7', 'eCC0', 'eCC1', 'eCC2', 'eCC7', 'fitness']
        # 將 dictionary 寫入 CSV 檔
        writer = csv.DictWriter(csvfile, fieldnames=colname)
        # 寫入第一列的欄位名稱
        writer.writeheader()
        for ind in pop:
            temp = [i for i in ind]
            temp.insert(0,g)
            temp.append(ind.fitness.values[0])
            writer.writerow({k:i for k, i in zip(colname, temp)})
    
    length = len(pop)
    mean = sum(fits) / length
    print("  Avg %s" % mean)
    print("  Min %s" % min(fits))
    best = pop[np.argmin([ind.fitness.values[0] for ind in pop])]
    terminal = best.fitness.values[0]

In [ ]:
stats

In [ ]:
sns.set()
_ = plt.plot(range(1, len(stats)+1), [ s for s in stats['mu'] ], marker='.')
_ = plt.plot(range(1, len(stats)+1), [ s for s in stats['min'] ], marker='.')
_ = plt.title('Average & Minimum Fitness per Iteration')
_ = plt.xlabel('Iterations')
_ = plt.ylabel('Mape (%)')
_ = plt.legend(labels=['average mape value', 'min mape value'])
plt.show()